# Treinando modelo de regressão logística com SDK do Azure para Python

## Verificar o pacote **azure-ai-ml**

É necessário a última versão dos pacotes **azure-ai-ml** e **mlflow** para rodar o código nesse notebook. Executar as células abaixo para verificar se está na última versão.

> **Obs**:
> Se os pacotes **azure-ai-ml** e **mlflow** não estiver instalado, execute `pip install azure-ai-ml` e `pip install mlflow azureml-mlflow` para instalar.

In [ ]:
pip show azure-ai-ml

In [ ]:
pip show mlflow azureml-mlflow

## Conectar workspace do Azure Machine Learning

Para conectar a workspace é preciso ter antes criado os seguintes itens: Resource Group > Workspace Azure Machine Learning > Instância de Computação e Cluster.

No script será necessário identificar os parâmetros, mas ao executar esse notebook dentro de um Studio do Azure Machine Learning, esses podem ser setados como Default.

In [26]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Verifica se consegue gerar token
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Em caso de falha, o Browser pedirá as credenciais.
    credential = InteractiveBrowserCredential()

In [ ]:
# Config MLClient
ml_client = MLClient.from_config(credential=credential)

## Treinando o Modelo

Para treinar um modelo, é preciso criar o script python (**vendas-sorvetes.py**) na pasta **src**. O script utilizará o arquivo **vendas_sorvetes_clima_itanhaem_2023.csv** da mesma pasta.

In [ ]:
%%writefile src/vendas-sorvetes-2.py
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Dados
df = pd.read_csv('vendas_sorvetes_clima_itanhaem_2023.csv')
df['ocorrencia_chuva'] = df['ocorrencia_chuva'].map({'Sim': 1, 'Não': 0})
X, y = df[['temp_media', 'precipitacao', 'ocorrencia_chuva']].values, df['Qtd'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Modelo
model = LinearRegression().fit(X_train, y_train)
y_hat = model.predict(X_test)

# Métricas
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)

mlflow.log_metric("MAE", mae)
mlflow.log_metric("MSE", mse)
mlflow.log_metric("RMSE", rmse)
mlflow.log_metric("R2", r2)

# Gráfico
residuos = y_test - y_hat
plt.scatter(y_hat, residuos)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predições')
plt.ylabel('Resíduos')
plt.title('Gráfico de Resíduos')

os.makedirs("outputs", exist_ok=True)
plt.savefig("outputs/residuos.png")
mlflow.log_artifact("outputs/residuos.png", artifact_path="figuras")



Executar a célula abaixo para gerar um job de execução do script:

In [ ]:
from azure.ai.ml import command

# configurar o job
job = command(
    code="./src",
    command="python vendas-sorvetes-2.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="computeml",
    display_name="sorvetes-pythonv2-treino",
    experiment_name="vendas-sorvetes-reglinear"
)

# criar job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)